In [124]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import os
import sys
from PIL import Image
import pandas as pd
import numpy as np
import seaborn as sns
import scipy
import keras
import tensorflow as tf
import cv2
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.metrics import classification_report,confusion_matrix
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from sklearn.utils import shuffle
from tensorflow.keras.optimizers import Adam

from tqdm import tqdm
np.random.seed(0)
tf.compat.v1.random.set_random_seed(0)

In [208]:
directory = os.path.join(os.getcwd(), 'data', 'dataset')
fake_directory = os.path.join(directory, 'fake')
real_directory = os.path.join(directory, 'real')

fake = []
real = []

In [126]:
for name in tqdm(os.listdir(fake_directory)):
  fake.append(os.path.join(fake_directory, name))

for name in tqdm(os.listdir(real_directory)):
  real.append(os.path.join(real_directory, name))

100%|██████████| 1000/1000 [00:00<00:00, 200052.66it/s]


In [200]:
y = np.ones(len(real))
x = np.zeros(len(fake))

label = np.concatenate([y, x])

names_list = real + fake

new_data, new_label = shuffle(names_list, label,  random_state = 0)

size = len(new_data)
train_size = int(size * 0.8) * -1
test_size = int(size * 0.2) * -1

train, test, y_train, y_test = new_data[:test_size], new_data[train_size:], new_label[:train_size], new_label[test_size:]

400


In [222]:
def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.uint8)
    img = tf.image.resize(img, [224, 224])
    return img

def get_label(file_path):
    cat = tf.strings.split(file_path, ':')[1]
    print(cat)
    if cat == b'fake':
        return 1
    return 0

def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [223]:
num_threads, num_epochs, train_len = 8, 20, len(train) 
train_ds = tf.data.Dataset.from_tensor_slices(train)
train_ds = train_ds.map(process_path, num_parallel_calls=tf.data.experimental.AUTOTUNE)

Tensor("strided_slice:0", shape=(), dtype=string)


In [224]:
train_ds = train_ds.shuffle(train_len)
train_ds = train_ds.batch(64)
train_ds = train_ds.prefetch(1)

In [225]:
from tensorflow.keras.applications.vgg16 import VGG16

model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [226]:
opt = tf.keras.optimizers.Adam(
    learning_rate= 0.01,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False)

def loss_fn(labels, predictions):
    return tf.math.confusion_matrix(
    labels, predictions, num_classes=2, weights=None, dtype=tf.dtypes.int32,
    name=None).numpy()[0,1]

tp = tf.keras.metrics.TruePositives(thresholds=None, name=None, dtype=None)
tn = tf.keras.metrics.TrueNegatives(thresholds=None, name=None, dtype=None)
fp = tf.keras.metrics.FalsePositives(thresholds=None, name=None, dtype=None)
fn = tf.keras.metrics.FalseNegatives(thresholds=None, name=None, dtype=None)

In [228]:
classifier = Sequential()
classifier.add(model)
classifier.add(Flatten())
classifier.add(Dense(1024, activation='relu'))
classifier.add(Dense(1, activation = 'sigmoid'))
classifier.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy', tp,tn,fp,fn])

In [229]:
history = classifier.fit(train_ds, epochs = 20, shuffle=True, batch_size=32)

Epoch 1/20
25/25 [==============================] - 42s 713ms/step - loss: 0.6502 - accuracy: 0.9619 - true_positives_17: 0.0000e+00 - true_negatives_17: 1539.0000 - false_positives_17: 61.0000 - false_negatives_17: 0.0000e+00
Epoch 2/20
25/25 [==============================] - 20s 717ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - true_positives_17: 0.0000e+00 - true_negatives_17: 1600.0000 - false_positives_17: 0.0000e+00 - false_negatives_17: 0.0000e+00
Epoch 3/20
25/25 [==============================] - 20s 720ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - true_positives_17: 0.0000e+00 - true_negatives_17: 1600.0000 - false_positives_17: 0.0000e+00 - false_negatives_17: 0.0000e+00
Epoch 4/20
25/25 [==============================] - 20s 720ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - true_positives_17: 0.0000e+00 - true_negatives_17: 1600.0000 - false_positives_17: 0.0000e+00 - false_negatives_17: 0.0000e+00
Epoch 5/20
25/25 [==============================] - 20s 721ms/step - lo

In [237]:
df = pd.DataFrame()
for idx, key in enumerate(history.history.keys()):
    row = pd.DataFrame(data={
      'item': key,
      'max': max(history.history[key]),
      'mean': np.mean(history.history[key]),
      'min': min(history.history[key]),
    }, index=[idx])
    df = pd.concat([df,row])

from IPython.display import HTML
df.sort_values(by=['item'],ascending=True)
HTML(df.to_html(index=False))

item,max,mean,min
loss,0.650192,0.032510,0.000000
accuracy,1.000000,0.998094,0.961875
true_positives_17,0.000000,0.000000,0.000000
true_negatives_17,1600.000000,1596.950000,1539.000000
false_positives_17,61.000000,3.050000,0.000000
false_negatives_17,0.000000,0.000000,0.000000
